# Predict pollution

## Paramters

In [39]:
from pathlib import Path

BASE_DIR = Path('/Users/efraflores/Desktop/EF/Diplo/data/04/pollution')
FILE_NAME = 'contaminantes.csv'

## Import

In [40]:
import pandas as pd

df = pd.read_csv(BASE_DIR.joinpath(FILE_NAME))
df.sample()

,id,fecha,nom_estac,clave_de_estacion,clave_parametro,nombre_de_parametro,unidades_parametros,valor,id_unidad,clave_de_unidad,nombre_de_unidad,alt,obs_estac,id_station,latitud,longitud,coordenadas
130978,130978,2020-02-27T08:00:00+00:00,Cuautitlán,CUT,PM10,Partículas menores a 10 micrómetros,2,65.0,2,µg/m³,microgramos por metro cúbico,2263,NaN,484150950109,19.722186,-99.198602,"19.722186,-99.198602"


## Transform

### Date variables

In [41]:
df['fecha'] = pd.to_datetime(df['fecha']).dt.tz_convert('America/Mexico_City')
df['date'] = df['fecha'].dt.date.astype(str)
df['hour'] = df['fecha'].dt.hour.astype(str).str.zfill(2)
df.sample()

,id,fecha,nom_estac,clave_de_estacion,clave_parametro,nombre_de_parametro,unidades_parametros,valor,id_unidad,clave_de_unidad,nombre_de_unidad,alt,obs_estac,id_station,latitud,longitud,coordenadas,date,hour
182706,182706,2020-07-02 12:00:00-05:00,Milpa Alta,MPA,PM10,Partículas menores a 10 micrómetros,2,NaN,2,µg/m³,microgramos por metro cúbico,2594,NaN,484090090104,19.1769,-98.990189,"19.1769,-98.990189",2020-07-02,12


### Which station and param?

In [42]:
station_param = []
for station in set(df['clave_de_estacion']):
    same_station = df[df['clave_de_estacion']==station].copy()
    for param in set(same_station['clave_parametro']):
        same_param = same_station[same_station['clave_parametro']==param].copy()
        tot_dates = pd.DataFrame(pd.date_range(start=same_param['fecha'].dt.date.min(), end=same_param['fecha'].dt.date.max()), columns=['date'])
        non_registered = tot_dates.astype(str).merge(same_param.groupby('date')['valor'].sum().reset_index(), how='left')['valor'].isnull()
        station_param.append((station, param, non_registered.sum(), non_registered.mean()))

In [43]:
spn = pd.DataFrame(station_param, columns=['station','param','null','%_null'])
spn

,station,param,null,%_null
0,ACO,O3,236,0.696165
1,ACO,CO,236,0.696165
2,ACO,NO2,236,0.696165
3,ACO,SO2,236,0.696165
4,ACO,PM10,236,0.696165
...,...,...,...,...
210,CUA,CO,236,0.696165
211,CUA,NO2,236,0.696165
212,CUA,SO2,236,0.696165
213,CUA,PM10,236,0.696165


In [44]:
spn['null'].describe()

count    215.0
mean     236.0
std        0.0
min      236.0
25%      236.0
50%      236.0
75%      236.0
max      236.0
Name: null, dtype: float64

### Choose station and parameter

In [45]:
df = df[(df['clave_de_estacion']=='SFE')&(df['clave_parametro']=='NO2')].reset_index(drop=True)
df.sample()

,id,fecha,nom_estac,clave_de_estacion,clave_parametro,nombre_de_parametro,unidades_parametros,valor,id_unidad,clave_de_unidad,nombre_de_unidad,alt,obs_estac,id_station,latitud,longitud,coordenadas,date,hour
598,134631,2020-02-15 12:00:00-06:00,Santa fe,SFE,NO2,Dióxido de nitrógeno,1,16.0,1,ppb,partes por billón,2599,NaN,484090040309,19.357357,-99.262865,"19.357357,-99.262865",2020-02-15,12


### All dates

In [46]:
df = pd.DataFrame(pd.date_range(start=df['fecha'].dt.date.min(), end=df['fecha'].dt.date.max()), columns=['date']).astype(str).merge(df, how='left')
df.isnull().sum()

date                      0
id                      236
fecha                   236
nom_estac               236
clave_de_estacion       236
clave_parametro         236
nombre_de_parametro     236
unidades_parametros     236
valor                   333
id_unidad               236
clave_de_unidad         236
nombre_de_unidad        236
alt                     236
obs_estac              2420
id_station              236
latitud                 236
longitud                236
coordenadas             236
hour                    236
dtype: int64

### Date by hour

In [47]:
df['hour'] = df['hour'].fillna('00')

In [48]:
df = df.pivot_table(index='date', columns='hour', values='valor', aggfunc='sum').melt(ignore_index=False).reset_index()
df.sort_values(['date','hour'], inplace=True)
df.isnull().sum()

date        0
hour        0
value    5716
dtype: int64

In [49]:
import cufflinks as cf
cf.go_offline()
df.set_index(['date','hour']).iplot()

In [50]:
df.groupby('date')['value'].sum().iplot()